# Relative Notebook Imports

As we know

In [1]:
    try: from . import relative
    except ImportError: ...

When notebooks __import__ as valid source then a relative __import__ has meaning.  The relative __import__ operates in two contexts:

#### 1. The interactive context when `__name__ == '__main__'`


#### 2. The __import__ed context when `__name__ == Path(__file__).stem`

## Usage

Relative is an IPython magic that works with anything in the `sys.meta_path`.

    %reload_ext rites.relative

In [2]:
from ast import parse, NodeTransformer, copy_location
blank = parse("""try: from . import object
except: import object""").body[0]

In [11]:
class Relative(NodeTransformer):
    def visit_ImportFrom(Relative, node):
        if node.module: return node
        if node.level is 1:
            blank.body[0].names = blank.handlers[0].body[0].names = node.names
            return copy_location(blank, node)
        if node.level is 2:"""Make a context manager for the meta_path that can jump up a directory."""
        return node

In [9]:
def unload_ipython_extension(ip=get_ipython()): return [
    object for object in ip.ast_transformers if not isinstance(object, Relative)]

def load_ipython_extension(ip=get_ipython()): ip.ast_transformers = unload_ipython_extension(ip) +[Relative()]
    
if __name__ == '__main__':
    %reload_ext rites
    load_ipython_extension()

In [5]:
from . import relative
if __name__ == '__main__':
    %reload_ext testing

In [6]:
""">>> assert relative is relative.relative"""
""">>> assert relative.__file__.endswith('.ipynb')"""
""">>> assert isinstance(relative, __import__('types').ModuleType)"""

">>> assert isinstance(relative, __import__('types').ModuleType)"

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


In [7]:
if __name__ == '__main__':
    %unload_ext testing